In [1]:
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt
# from imageai import Detection

In [2]:
class FPS:
    def __init__(self):
        self.pTime = time.time()
    def update(self, img=None, pos=(20, 50), color=(255, 0, 0), scale=3, thickness=3):
        cTime = time.time()
        try:
            fps = 1 / (cTime - self.pTime)
            self.pTime = cTime
            if img is None:
                return fps
            else:
                cv2.putText(img, f"FPS: {int(fps)}", pos, cv2.FONT_HERSHEY_PLAIN, scale, color, thickness)
                return fps, img
        except:
            return 0

In [3]:
cap = cv2.VideoCapture(0)
_, image = cap.read()
HEIGHT, WIDTH, _ = image.shape
cap.release()

In [4]:
classes = None
with open('./yolo/coco.names', 'rt') as f:
    classes = [line.strip() for line in f.readlines()]

In [5]:
configPath = './yolo/yolov3-tiny.cfg'
weightsPath = './yolo/yolov3-tiny.weights'

In [6]:
net = cv2.dnn_DetectionModel(weightsPath, configPath)

In [9]:
net.setInputSize(width=WIDTH//5, height=HEIGHT//5)
net.setInputScale(1.0 / 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

<dnn_Model 00000183867E7530>

In [10]:
fps = FPS()
cap = cv2.VideoCapture(0)

while True:
    try:
        _,img = cap.read()
        fps.update(img)
        height, width, channels = img.shape

        # Detecting objects
        classIds, confs, bbox = net.detect(img, confThreshold = 0.5)
        # print(classIds, bbox)

        bbox = list(bbox)
        confs = list(np.array(confs).reshape(1, -1)[0])
        # confs = list(map(float, confs))

        indices = cv2.dnn.NMSBoxes(bbox, confs, 0.5, 0.2)
        # print(indices)

        for i in indices:
            box = bbox[i]
            x, y, w, h = box[0], box[1], box[2], box[3]
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.putText(img, f'{classes[classIds[i]].upper()} {confs[i]}', box[:2], cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

        # if len(classIds) > 0:
        #     for classId, conf, box in zip(classIds.flatten(), confs.flatten(), bbox):
        #         cv2.rectangle(img, box, color=(0, 255, 0), thickness=2)
        #         cv2.putText(img, f'{classes[classId].upper()} {conf}', box[:2], cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

        cv2.imshow("Image", img)
        key = cv2.waitKey(1)
        if key == 27:
            break
    except Exception as e:
        print(e)
        break


cap.release()
cv2.destroyAllWindows()